In [11]:
import sys
sys.path.append('../src')
import numpy as np
from preprocessing import prepare_dl_data

import pandas as pd
from sklearn.model_selection import train_test_split

import os

data_path = '../data_dl/train'
X, y = prepare_dl_data(data_path)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

save_dir = '../data_dl'
os.makedirs(save_dir, exist_ok=True)

#np.save(f"{save_dir}/X_train.npy", X_train)
#np.save(f"{save_dir}/X_test.npy", X_test)
#np.save(f"{save_dir}/Y_train.npy", y_train)
#np.save(f"{save_dir}/Y_test.npy", y_test)

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
class CatDogDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).permute(0, 3, 1, 2).float()
        self.y = torch.tensor(y).long()

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = CatDogDataset(X_train, y_train)
val_dataset   = CatDogDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32)

NameError: name 'X_train' is not defined

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
pip install timm

  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 13.2 MB/s  0:00:00 eta 0:00:01
Using cached fsspec-2025.10.0-py3-none-any.whl (200 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.7.1
    Uninstalling fsspec-2022.7.1:
      Successfully uninstalled fsspec-2022.7.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [timm]6/7 [timm]ngface_hub]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import timm
pretrained_model = timm.create_model(
    'vit_base_patch16_224',
    pretrained=True,
    num_classes=2 
)
pretrained_model = pretrained_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(pretrained_model.parameters(), lr=1e-4, weight_decay=0.01)

In [ ]:
epochs = 3

for epoch in range(epochs):
    pretrained_model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = pretrained_model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += y_batch.size(0)
        train_correct += predicted.eq(y_batch).sum().item()
    
    pretrained_model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = pretrained_model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += y_batch.size(0)
            val_correct += predicted.eq(y_batch).sum().item()
    
    train_acc = 100. * train_correct / train_total
    val_acc = 100. * val_correct / val_total
    
    print(f'Epoch {epoch+1}/{epochs} - '
          f'Train Loss: {train_loss/len(train_loader):.4f}, '
          f'Train Acc: {train_acc:.2f}%, '
          f'Val Loss: {val_loss/len(val_loader):.4f}, '
          f'Val Acc: {val_acc:.2f}%')

In [ ]:
pretrained_model.eval()
with torch.no_grad():
    X_batch, y_batch = next(iter(val_loader))
    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
    outputs = pretrained_model(X_batch)
    _, predicted = outputs.max(1)
    probs = torch.softmax(outputs, dim=1)
    
    print("\nSample predictions:")
    for i in range(10):
        true_label = 'cat' if y_batch[i] == 0 else 'dog'
        pred_label = 'cat' if predicted[i] == 0 else 'dog'
        confidence = probs[i][predicted[i]].item() * 100
        print(f'True: {true_label}, Pred: {pred_label}, Confidence: {confidence:.2f}%')